In [ ]:
from pathlib import Path
import json
from sqlalchemy import create_engine
from log_pipeline import LogPipeline

# config yükle
cfg_text = Path("//Users//sdedeoglu//Desktop//python//config.json").read_text(encoding="utf-8")
cfg = json.loads(cfg_text)
engine = create_engine(f"mysql+pymysql://{cfg['kullanici']}:{cfg['sifre']}@{cfg['host']}:{cfg['port']}/{cfg['veritabani']}")

pipeline = LogPipeline("//Users//sdedeoglu//Desktop//python//case_data.parquet.gzip", engine)
pipeline.run_once()

#yada

In [ ]:
from pathlib import Path
import json
from sqlalchemy import create_engine
from log_pipeline import LogPipeline

# config yükle
cfg_text = Path("/Users/sdedeoglu/Desktop/python/config.json").read_text(encoding="utf-8")
cfg = json.loads(cfg_text)
engine = create_engine(f"mysql+pymysql://{cfg['kullanici']}:{cfg['sifre']}@{cfg['host']}:{cfg['port']}/{cfg['veritabani']}")

pipeline = LogPipeline("/Users/sdedeoglu/Desktop/python/case_data.parquet.gzip", engine)

# Büyük dosyalar için chunk
for batch in pipeline.reader.read_in_batches(batch_size=10000):
    # zorunlu sütunları temizle
    batch.dropna(subset=["user_id", "session_id", "hotel_id", "request_id", "funnel_id"], inplace=True)
    users, sessions, events, hotels, payments = pipeline.validator.validate(batch)

    # updated_Date ekle
    now = pipeline.now
    for tbl in (users, sessions, events, hotels, payments):
        if not getattr(tbl, "empty", True):
            tbl["updated_Date"] = now

    # yaz
    pipeline.writer.upsert_df(users, "users")
    pipeline.writer.upsert_df(sessions, "sessions")
    pipeline.writer.upsert_df(events, "events")
    pipeline.writer.upsert_df(hotels, "hotels")
    pipeline.writer.upsert_df(payments, "payments")